In [1]:
import pandas as pd
dataset = pd.read_excel('Dataset/Bank_Personal_Loan_Modelling.xlsx',sheet_name='Data')

In [2]:
dataset.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [3]:
dataset1=dataset.drop(['ID', 'ZIP Code'], axis=1)
dataset1.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [4]:
dataset2 = dataset1.dropna()
dataset3 = dataset2.drop_duplicates()

In [5]:
import numpy as np
dataset3["CCAvg"] = np.round(dataset3["CCAvg"])


C:\Users\Joy\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)

features = ['Age','Experience','Income','Family','CCAvg','Education','Mortgage','Securities Account','CD Account','Online','CreditCard']

rf_model.fit(X=dataset3[features],y=dataset3["Personal Loan"])

RandomForestClassifier(max_features=2, n_estimators=1000, oob_score=True)

In [7]:
print("OOB Accuracy score: ", rf_model.oob_score_)

OOB Accuracy score:  0.986164026468819


#### Sorting out the Important variables

In [8]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature, "    \t:", imp)

Age     	: 0.05099975756395016
Experience     	: 0.05118737974909637
Income     	: 0.36065658614213125
Family     	: 0.10389075637715041
CCAvg     	: 0.1385979019421533
Education     	: 0.1651343128333275
Mortgage     	: 0.047771058406858885
Securities Account     	: 0.006374218620408789
CD Account     	: 0.05468560914866897
Online     	: 0.009456695136869224
CreditCard     	: 0.011245724079385079


#### The variables with higher importance value :
##### ______1. income
##### ______2. CCAvg
##### ______3. Education   

#### Taking these variables to fit in the model 

In [9]:
from sklearn import tree

tree_model = tree.DecisionTreeClassifier(max_depth=6, max_leaf_nodes=12)

cl_data = pd.DataFrame([dataset3["Education"],dataset3["CCAvg"],dataset3["Income"]]).T

tree_model.fit(X=cl_data,y=dataset3["Personal Loan"])

DecisionTreeClassifier(max_depth=6, max_leaf_nodes=12)

In [10]:
with open("Dtree_bank_loan.dot","w") as f:
    f=tree.export_graphviz(tree_model,feature_names=["Education","CCAvg","Income"],out_file=f)

### Rules for Loan Sanctions

##### 1. Person having income more than 106.5, with CCAvg more than 2.5 have a high chance of getting the loan.
##### 2. Person having income more than 88.5, with Education level 2 or more have a high chance of getting the loan.
##### 3. Person having income more than 92.5, with Education level 3 or more have a high chance of getting the loan.
##### 4. Person having income more than 116.5, with Education level 2 or more have a high chance of getting the loan.
##### 5. Person having income more than 116.5, with CCAvg more than 3.5 have a high chance of getting the loan.